In [1]:
import pandas as pd
# pd.get_option("display.max_columns")
pd.set_option("display.max_columns", None)

In [2]:
import wrds

In [3]:
conn = wrds.Connection()

Enter your WRDS username [n2ngo]: tiwata
Enter your password: ·············


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
Loading library list...
Done


In [123]:
# conn.list_libraries()
conn.list_tables(library='crsp_a_stock')

['dse',
 'dseall',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',
 'dsfhdr',
 'dsi',
 'dsiy',
 'mse',
 'mseall',
 'msedelist',
 'msedist',
 'mseexchdates',
 'msenames',
 'msenasdin',
 'mseshares',
 'msf',
 'msfhdr',
 'msi',
 'msiy',
 'saz_agg_ann',
 'saz_agg_mth',
 'saz_agg_qtr',
 'saz_del',
 'saz_dind',
 'saz_dis',
 'saz_dp_dly',
 'saz_ds_dly',
 'saz_hdr',
 'saz_indhdr',
 'saz_mdel',
 'saz_mind',
 'saz_mth',
 'saz_nam',
 'saz_ndi',
 'saz_shr',
 'stock_qvards',
 'stocknames']

In [6]:
isin_rri =  conn.get_table(library='reprisk', table= 'v2_wrds_company_lookup',obs=10)
isin_rri

,reprisk_id,company_name,isin
0,10,Acer Inc,TW0002353000
1,10,Acer Inc,US0044341065
2,10,Acer Inc,US0044342055
3,100,Rio Tinto PLC,ARDEUT112638
4,100,Rio Tinto PLC,BRRIOTBDR007
5,100,Rio Tinto PLC,GB0007188757
6,100,Rio Tinto PLC,GB0007406639
7,100,Rio Tinto PLC,US7672041008
8,1000,Terrane Metals Corp,CA88103A1084
9,1000,Terrane Metals Corp,CA88103A1167


In [5]:
rri = conn.get_table(library='reprisk', table='pm_rri_data',obs=10)
rri

,reprisk_id,date,current_rri,rri_trend,peak_rri,peak_rri_date,reprisk_rating,country_sector_average,environmental_percentage,social_percentage,governance_percentage
0,10,2007-01-31,21.0,-1.0,41.0,2006-08-25,BBB,13.0,1.00,0.00,0.0
1,10,2007-02-28,34.0,13.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
2,10,2007-03-31,27.0,-7.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
3,10,2007-04-30,24.0,-3.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
4,10,2007-05-31,23.0,-1.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
5,10,2007-06-30,22.0,-1.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
6,10,2007-07-31,20.0,-2.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
7,10,2007-08-31,27.0,7.0,41.0,2006-08-25,BBB,13.0,0.67,0.33,0.0
8,10,2007-09-30,29.0,2.0,41.0,2006-08-25,BBB,13.0,0.50,0.50,0.0
9,10,2007-10-31,25.0,-4.0,41.0,2006-08-25,BBB,13.0,0.50,0.50,0.0


index constituents

In [8]:
list_index = conn.get_table(library='comp_global', table='g_idx_index')
list_index.to_csv("list_index.csv",index=False)

In [49]:
# S&P Global 1200 Index
sp1200_hist = conn.raw_sql("select * from comp_global.g_idxcst_his where gvkeyx = '150918'")
sp1200_hist.to_csv("spglobal1200_hist.csv",index=False)

In [33]:
sp1200_hist.head()

,gvkey,iid,gvkeyx,from,thru
0,001932,01W,150918,1989-09-11,None
1,001945,01W,150918,1989-09-11,2006-09-04
2,002000,01W,150918,1989-09-11,1999-04-16
3,002338,01W,150918,1989-09-11,1998-04-20
4,002338,01W,150918,2005-06-30,2016-06-20


In [15]:
from tqdm import tqdm

In [69]:
ids = pd.DataFrame()
for gvkey in tqdm(sp1200_hist.gvkey):
    tmp = conn.raw_sql("select * from comp_global.g_security where gvkey = '"+str(gvkey)+"'")
    ids = pd.concat([ids,tmp],axis=0)

100%|██████████| 1480/1480 [03:36<00:00,  6.84it/s]


In [71]:
ids.to_csv("spglobal1200_ids.csv",index=False)

## RRI

In [4]:
ids = pd.read_csv("spglobal1200_ids.csv")

In [10]:
import numpy as np

In [17]:
repriskids = pd.DataFrame()
for isin in tqdm(ids['isin'].unique()):
    if type(isin)==str:
        tmp = conn.raw_sql("select * from reprisk.v2_wrds_company_lookup where isin = '"+isin+"'")
        repriskids = pd.concat([repriskids,tmp],axis=0)

100%|██████████| 2208/2208 [05:16<00:00,  6.97it/s]


In [13]:
isin='GB0002875804'


,reprisk_id,company_name,isin
0,22,British American Tobacco PLC (BAT),GB0002875804


In [ ]:
financial data
-the firm’s total assets, 
-the firm’s ebitover-total-assets ratio,

In [81]:
conn.get_table(library='comp', table='g_funda',
               # columns = ['conm','isin','sedol','fyr',
               #                             'at','ebit'],
               obs=5)

,gvkey,indfmt,datafmt,consol,popsrc,acctstd,acqmeth,bspr,compst,curcd,final,fyear,fyr,ismod,pddur,scf,src,stalt,upd,datadate,fdate,pdate,accli,acco,aco,acofs,acox,acoxfs,acqdisn,acqdiso,act,adpac,am,amdc,ao,aoloch,aox,ap,apalch,apch,apdpfs,apfs,apo,apofs,aqc,artfs,asdis,asinv,at,atoch,autxr,bcef,bct,ca,capcst,capfl,capr1,capr2,capr3,caprt,caps,capx,capxfi,ceq,cfbd,cfere,cflaoth,cfo,cfpdo,cga,ch,che,cheb,chech,chee,chefs,chenfd,chfs,chs,cmp,cogs,crvnli,cshr,cstk,custadv,dbtb,dbte,dc,dcsfd,dcufd,dd1,dd1fs,dfpac,dfxa,dispoch,dlc,dlcch,dlcfs,dltis,dltr,dltt,do,doc,dp,dpact,dpc,dpdc,dpltb,dpsc,dpstb,dptb,dptc,dptic,dv,dvc,dvp,dvpdp,dvrec,dvrre,dvsco,dvt,ea,ebit,ebitda,eiea,eieac,emp,eqdivp,ero,exre,exres,exreu,fatb,fate,fatl,fatp,fca,fdfr,fea,fel,ffs,fiao,fincf,fininc,finle,finre,finvao,fopo,fsrco,fsrcopo,fsrcopt,fsrct,fuseo,fuset,gdwl,iaeq,iafxi,ialoi,ialti,iamli,iaoi,iapli,iarei,iassi,iasti,iati,ib,ibc,ibki,ibmii,icapt,idiis,idilb,idilc,idis,idist,idit,idits,iire,initb,intan,intand,intanp,intc,intfact,intfl,intiact,intoact,intpd,intpn,intrc,invch,invdsp,invfg,invo,invrm,invsvc,invt,invtfs,invwip,iobd,ioi,iore,ip,ipti,isgr,isgt,isgu,isoth,ist,ivaco,ivaeq,ivao,ivch,ivgod,ivi,ivncf,ivpt,ivst,ivstch,ivstfs,lcabg,lcacl,lcacr,lcag,lcal,lcalt,lcam,lcao,lcast,lcat,lco,lcofs,lcox,lct,lctfs,lcuacu,liqresn,liqreso,lndep,lninc,lnmd,lnrep,lo,lse,lt,ltdch,ltdlch,ltlo,mib,mibn,mibt,mic,mii,miseq,mtl,ncfliq,neqmi,nio,nit,noasub,nopi,np,npanl,npaore,nparl,npat,npfs,oancf,oancfc,oancfd,oiadp,oibdp,onbalb,onbale,opprft,pacqp,pcl,pi,pliach,ppegt,ppent,prc,prodv,prosai,prstkc,prv,psfix,pstk,pstkn,pstkr,ptran,purtshr,pvon,pvt,radp,ragr,rari,rati,rawmsm,rcl,re,recch,recco,reccofs,rect,rectfs,rectr,rectrfs,revt,ris,rlri,rlt,rpag,rv,rvbci,rvbpi,rvbti,rveqt,rvlrv,rvri,rvsi,rvti,rvupi,rvutx,saa,sal,sale,sbdc,sc,sco,seq,shrcap,siv,spi,sppch,sppiv,ssnp,sstk,stbo,stfixa,stinv,stio,stkch,subdis,subpur,tdsg,tdst,teq,transa,tsca,tstk,tstlta,tx,txc,txdb,txdc,txdi,txditc,txo,txop,txp,txpd,txpfs,txt,txw,ui,unl,unnp,vpac,vpo,wcap,wcapch,wcapchc,wcapopc,wcaps,wcapsa,wcapsu,wcapt,wcapu,xacc,xaccfs,xago,xagt,xcom,xcomi,xdvre,xeqo,xi,xido,xidoc,xindb,xindc,xins,xinst,xint,xintd,xivi,xivre,xlr,xnitb,xobd,xoi,xopr,xopro,xore,xpp,xppfs,xpr,xrd,xrent,xs,xsga,xstf,xstfo,xstfws,xt,iid,exchg,isin,sedol,ajexi,curcdi,cshoi,cshpria,epsexcon,epsexnc,epsincon,epsinnc,icapi,nicon,ninc,pv,tstkni,conm,costat,fic,loc,naicsh,sich,rank,au,auop
0,001166,INDL,HIST_STD,C,I,DS,None,GO,None,NLG,Y,1996.0,12.0,1.0,12.0,10.0,23.0,None,3.0,1996-12-31,None,None,None,None,17.783,None,17.783,None,None,None,395.228,None,0.000,None,0.0,9.245,0.0,70.089,None,-19.273,None,None,0.0,None,0.000,None,None,None,611.980,0.0,0.0,None,None,None,None,None,None,None,None,None,164.852,81.439,None,178.075,None,None,0.0,None,None,None,57.358,70.609,58.801,-1.443,57.358,None,None,None,None,None,332.410,None,None,0.331,-12.875,None,None,0.0,None,None,22.870,None,None,30.834,None,88.641,1.728,None,None,None,87.975,0.0,None,30.834,166.431,30.834,None,None,None,None,None,None,None,10.947,None,None,None,None,None,None,None,0.0,101.662,132.496,0.0,0.000,4.140,None,-39.315,8.800,None,None,None,None,None,None,0.0,None,None,None,None,0.000,20.029,None,None,None,None,-0.074,None,None,None,None,None,None,0.000,None,None,None,None,None,None,None,None,None,None,None,68.402,68.402,None,None,363.850,None,None,None,None,None,None,None,None,None,0.000,None,0.0,0.0,0.0,None,0.0,0.0,None,NaN,None,-28.221,None,16.611,-21.762,68.091,None,140.370,None,77.430,None,None,None,None,None,None,None,None,None,None,0.0,0.0,0.0,0.000,None,None,-80.649,None,13.251,-2.014,None,None,None,None,None,None,None,None,None,None,None,83.888,None,8.080,242.618,None,None,None,None,None,None,None,None,0.0,611.980,336.105,3.949,None,None,97.800,None,97.800,19.114,19.114,0.0,None,None,None,0.00,0.00,None,0.0,65.771,None,None,None,None,None,50.377,None,None,101.662,132.496,None,None,None,0.0,None,91.994,0.0,383.183,216.752,0.0,0.0,None,0.0,0.000,2.804,0.0,0.0,0.0

In [88]:
fdat1 = pd.DataFrame()
for isin,sedol in tqdm(zip(ids['isin'],ids['sedol']),total=ids.shape[0]):
# for isin,sedol in tqdm(zip(ids['isin'][:5],ids['isin'][:5]),total=5):
    if isin != "" and sedol != "":
        tmp = conn.raw_sql("select conm,isin,sedol,fyear,fyr,at,ebit from comp.g_funda where isin = '"+str(isin)+"' or sedol = '" + str(sedol) +"'"  )
        fdat1 = pd.concat([fdat1,tmp],axis=0)
fdat1.to_csv("financial_data1.csv",index=False)

100%|██████████| 3640/3640 [09:01<00:00,  6.72it/s]


financial data
-the price-to-book ratio,   
-the firm’s total assets,   
-the firm’s ebitover-total-assets ratio,  
-the firm’s size  

In [94]:
fdat2 = pd.DataFrame()
for gvkey in tqdm(sp1200_hist.gvkey[:5]):
    tmp = conn.raw_sql("select gvkey,permno,cusip,ticker,adate,qdate,public_date,bm,mktcap from wrdsapps_finratio.firm_ratio where gvkey = '"+str(gvkey)+"'")
    fdat2 = pd.concat([fdat2,tmp],axis=0)
fdat2.to_csv("financial_data2.csv",index=False)

100%|██████████| 5/5 [00:00<00:00,  6.95it/s]


In [104]:
conn.raw_sql("select count(distinct(gvkey)) from wrdsapps_finratio.firm_ratio")

,count
0,20491


In [101]:
sp1200_hist.gvkey[-1:]

1479    324977
Name: gvkey, dtype: object

In [95]:
fdat2

,gvkey,permno,cusip,ticker,adate,qdate,public_date,bm,mktcap


In [126]:
conn.raw_sql("select * from crsp_a_stock.msf limit 10")

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,vol,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,-2.562500,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50000,-4.437500,-4.375000,1771.0,NaN,None,None,3680.0,1.0,1.0,-4.375000,0.25000,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25000,-4.375000,-3.250000,828.0,-0.257143,None,None,3680.0,1.0,1.0,-3.250000,0.25000,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25000,-4.437500,-4.437500,1078.0,0.365385,None,None,3680.0,1.0,1.0,-4.437500,0.12500,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00000,-4.312500,-4.000000,957.0,-0.098592,None,None,3793.0,1.0,1.0,-4.000000,0.25000,1986-04-30,-0.098592
5,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-05-30,-3.06250,-4.218750,-3.109375,1074.0,-0.222656,None,None,3793.0,1.0,1.0,-3.109375,0.09375,1986-05-30,-0.222656
6,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-06-30,-2.90625,-3.296875,-3.093750,1069.0,-0.005025,None,None,3793.0,1.0,1.0,-3.093750,0.06250,1986-06-30,-0.005025
7,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-07-31,-2.59375,-3.437500,-2.843750,1163.0,-0.080808,None,None,3793.0,1.0,1.0,-2.843750,0.06250,1986-07-31,-0.080808
8,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-08-29,-1.03125,-2.625000,-1.093750,3049.0,-0.615385,None,None,3793.0,1.0,1.0,-1.093750,0.06250,1986-08-29,-0.615385
9,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-09-30,-0.96875,-1.281250,-1.031250,3551.0,-0.057143,None,None,3793.0,1.0,1.0,-1.031250,0.06250,1986-09-30,-0.057143


In [125]:
# conn.raw_sql("select * from comp_global.g_secd where trfd > 1 limit 10 ")
ret = pd.DataFrame()
# for isin,sedol in tqdm(zip(ids['isin'],ids['sedol']),total=ids.shape[0]):
for isin,sedol in tqdm(zip(ids['isin'][:5],ids['sedol'][:5]),total=5):
    if isin != "" and sedol != "":
        tmp = conn.raw_sql("select * from crsp_a_stock.msf where isin = '"+str(isin)+"' or sedol = '" + str(sedol) +"'")
        ret = pd.concat([ret,tmp],axis=0)
ret.to_csv("crsp_return_data.csv",index=False)

  0%|          | 0/5 [00:00<?, ?it/s]


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "isin" does not exist
LINE 1: select * from crsp_a_stock.msf where isin = 'GB0002875804' o...
                                             ^

[SQL: select * from crsp_a_stock.msf where isin = 'GB0002875804' or sedol = '0287580']
(Background on this error at: https://sqlalche.me/e/14/f405)

In [42]:
fins = conn.get_table(library='wrdsapps_finratio', table='firm_ratio', 
                                columns = ['gvkey','permno','cusip','ticker','adate','qdate','public_date',
                                           'bm','mktcap'], obs=5)
fins

,gvkey,permno,cusip,ticker,adate,qdate,public_date,bm,mktcap
0,001000,25881.0,None,None,1969-12-31,1970-09-30,1971-01-31,0.427328,23.895000
1,001000,25881.0,00003210,AE,1970-12-31,1970-12-31,1971-02-28,0.431071,29.536875
2,001000,25881.0,00003210,AE,1970-12-31,1970-12-31,1971-03-31,0.431071,25.886250
3,001000,25881.0,00003210,AE,1970-12-31,1970-12-31,1971-04-30,0.431071,28.541250
4,001000,25881.0,00003210,AE,1970-12-31,1971-03-31,1971-05-31,0.431071,22.899375


-the firm’s beta, 

In [28]:
conn.list_libraries()

['compbd',
 'compgd',
 'compnad',
 'compsegd',
 'crspa',
 'evts',
 'issm',
 'nastraq',
 'wrds_lib_internal',
 'tass',
 'taqm_2017',
 'aha',
 'toyo',
 'levin',
 'ahasamp',
 'tresgsmp',
 'trsdcgs',
 'columnar',
 'trws',
 'lspd',
 'twoiq',
 'wcai',
 'csmar',
 'wind',
 'wqa',
 'tresg',
 'kld_histall',
 'taqm_2018',
 'taqm_2019',
 'fjc_litigation',
 'msrb_all',
 'ciqsamp_ratings',
 'tr_13f',
 'taqm_2016',
 'custom_jl',
 'risk',
 'centris',
 'block_all',
 'zacks',
 'tr_ibes',
 'boardex_trial',
 'audit',
 'comp_global',
 'comp_na_monthly_all',
 'emdb',
 'wrdsapps_evtstudy_lr',
 'auditsmp',
 'boardsmp',
 'calcbnch',
 'wrdsrpts_rep_usage',
 'comp_segments_hist',
 'public_all',
 'pwt_all',
 'wrdsapps_finratio_ccm',
 'wrdsapps_finratio_ibes_ccm',
 'wrdsapps_link_crsp_optionm',
 'etfg',
 'cboe_all',
 'wrdsapps_backtest_plus',
 'pacap',
 'ciqsamp_transcripts',
 'ciq',
 'tr_ds',
 'ppublica',
 'preqin',
 'crsp_a_stock',
 'ciq_pplintel',
 'fisdsamp',
 'windsmp',
 'rent',
 'compmcur',
 'contrib_ceo_tur

In [31]:
conn.list_tables(library='wrdsapps_backtest_plus')

['signals_raw_plus']

In [32]:
beta =  conn.get_table(library='wrdsapps_backtest_plus', table='signals_raw_plus', 
                       #          columns = ['gvkey','permno','cusip','ticker','adate','qdate','public_date',
                       #                     'mktcap','bm','roe','de_ratio','rd_sale','adv_sale'],
                       obs=5)
beta

,permno,fdate,ptb,capes,capei,capec,capef,btm,cr,dr,accrual,accrualpct,gprof,gprofg,oprof,noa,dnoa,sg,atg,cch,roa,roe,itoa,taxexp,deftax,pmg,div_p,cf_p,cf,cfm,aue,aueg,fcf_p,fcf,sale_p,pe,ev_ebitda,pm_p,rd_sale,rdg,rd_p,hire,secudebt,convdebt,ltdg,cash_to_debt,quick,current,cheturn,recturn,invturn,deprate,quickg,currentg,cheturng,recturng,invturng,deprateg,taxinc,saleg,capxg,capxag,capxltg,pmsg,sgag,invg,roag,drg,corpinvest,ceqg,oplev,revsup,firmtang,reholding,cashprod,pfs_score,cfvol,cfvols,oivol,roavol,accrualvol,absaccrual,geo_seg,ior,dbreadth,dior,dblock_n,dhhi,divyield,momentum,momentum12,momentum36,ltr36,ltr60,beta,betasq,ivol,maxret,seasonality,cei,nsi,chs,o_score,sue,numest,prcdelay1,prcdelay2,prcdelay3,dispersion,str_mod,best_idea,active_flow,skew_otm,skew_atm,smirk,rv_iv,pc_divergence,parity_vspread,pc_ratio,sir,sio,scr,liqvol,divinc_prob,split_prob,stockdiv_prob,specdiv_prob,trend_factor,coleman,avgwordsent,avgwordpara,ft_mstrong,fog_complex,smog_complex,filingsize,momaccel
0,10000.0,1986-06-30,-20.093708,None,None,None,None,0.039325,-8.964286,0.060164,0.067911,-0.077487,0.009177,None,-0.752524,-0.557969,None,None,None,None,-0.826196,-1.386067,None,None,None,None,0.0,-0.090928,-0.830997,-2.660848,0.312305,None,-0.105841,-0.96729,0.034172,-0.069879,None,0.000852,0.0,None,0.0,None,0.0,0.0,None,-1.247875,-0.729904,-1.210611,8.359494,67.387755,11.043478,0.701493,None,None,None,None,None,None,None,4.166667,None,None,None,None,None,-0.044594,None,NaN,-1.793450,0.161103,1.618834,0.010652,0.546046,0.0,-34.171835,2.0,None,None,None,None,None,-0.067911,None,None,None,None,None,None,None,-0.289286,None,None,None,None,NaN,NaN,-0.026370,-0.033998,None,None,None,6.042839,-3.739115,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,10000.0,1986-07-31,-18.453137,None,None,None,None,0.039325,-8.964286,0.065453,0.067911,-0.077487,0.009177,None,-0.752524,-0.557969,None,None,None,None,-0.826196,-1.386067,None,None,None,None,0.0,-0.098921,-0.830997,-2.660848,0.312305,None,-0.115146,-0.96729,0.037177,-0.076022,None,0.000927,0.0,None,0.0,None,0.0,0.0,None,-1.247875,-0.729904,-1.210611,8.359494,67.387755,11.043478,0.701493,None,None,None,None,None,None,None,4.166667,None,None,None,None,None,-0.044594,None,NaN,-1.689820,0.161103,1.618834,0.011589,0.546046,0.0,-34.171835,2.0,None,None,None,None,None,-0.067911,None,None,None,None,None,None,None,-0.292857,None,None,None,None,NaN,NaN,-0.037241,-0.093671,None,None,None,5.797454,-3.739115,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,10000.0,1986-08-31,-6.969141,None,None,None,None,0.039325,-8.964286,0.170178,0.067911,-0.077487,0.009177,None,-0.752524,-0.557969,None,None,None,None,-0.826196,-1.386067,None,None,None,None,0.0,-0.257196,-0.830997,-2.660848,0.312305,None,-0.299379,-0.96729,0.096659,-0.197657,None,0.002410,0.0,None,0.0,None,0.0,0.0,None,-1.247875,-0.729904,-1.210611,8.359494,67.387755,11.043478,0.701493,None,None,None,None,None,None,None,4.166667,None,None,None,None,None,-0.044594,None,NaN,-1.566740,0.161103,1.618834,0.030131,0.546046,0.0,-34.171835,2.0,None,None,None,None,None,-0.067911,None,None,None,None,None,None,None,-0.350000,None,None,None,None,NaN,NaN,-0.071863,-0.064992,None,None,None,4.902429,-3.739115,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,10000.0,1986-09-30,-13.124567,None,None,None,None,0.027096,-9.547619,0.261279,0.083843,-0.093842,0.010480,None,-0.753712,-0.493450,None,None,None,None,-0.791985,-1.614838,None,None,None,None,0.0,-0.279175,-0.832952,-2.696296,0.308924,None,-0.325959,-0.97254,0.103540,-0.220630,None,0.003068,0.0,None,0.0,None,0.0,0.0,None,-0.907045,-0.587420,-0.855011,0.952941,6.639344,1.613546,0.494845,None

In [30]:
beta =  conn.get_table(library='wrdsapps_backtest_basic', table='mastertable', 
                       #          columns = ['gvkey','permno','cusip','ticker','adate','qdate','public_date',
                       #                     'mktcap','bm','roe','de_ratio','rd_sale','adv_sale'],
                       obs=5)
beta

,permno,date,invsigma2,ret,mktcap,retx,rf,dgtw_xret,b_mkt,b_smb,b_hml,b_umd,ffxret,gvkey,ffi48,ffi48_desc,sp500_weight,crsp_weight,wrds_lc_500,wrds_mc_500,wrds_sc_2000,wrds_micro,wrds_index
0,10000.0,1986-06-30,913.669487,-0.005025,11.734594,-0.005025,0.0052,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000004,None,None,None,0.000300,0.000005
1,10000.0,1986-07-31,804.226220,-0.080808,10.786344,-0.080808,0.0052,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000004,None,None,None,0.000300,0.000005
2,10000.0,1986-08-31,685.662587,-0.615385,4.148594,-0.615385,0.0046,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000002,None,None,None,0.000116,0.000002
3,10000.0,1986-09-30,794.590129,-0.057143,3.911531,-0.057143,0.0045,None,None,None,None,None,None,013007,9.0,Hshld,None,0.000002,None,None,None,0.000114,0.000002
4,10000.0,1986-10-31,749.217156,-0.242424,3.002344,-0.242424,0.0046,None,None,None,None,None,None,013007,6.0,Toys,None,0.000001,None,None,None,0.000085,0.000001


isin and gvkey